# Mask R-CNN - Train FCN using MRCNN in Predict Mode 

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os, sys, math, io, time, gc, argparse, platform, pprint
import numpy as np
import tensorflow as tf
import keras
import keras.backend as KB
sys.path.append('../')
import mrcnn.model_mrcnn  as mrcnn_modellib
import mrcnn.model_fcn    as fcn_modellib
import mrcnn.visualize    as visualize
import mrcnn.new_shapes   as shapes
from datetime import datetime   
from mrcnn.utils        import command_line_parser, Paths
from mrcnn.config       import Config
from mrcnn.dataset      import Dataset 
from mrcnn.utils        import log, stack_tensors, stack_tensors_3d, write_stdout
from mrcnn.datagen      import data_generator, load_image_gt
from mrcnn.callbacks    import get_layer_output_1,get_layer_output_2
from mrcnn.coco         import CocoDataset, CocoConfig, CocoInferenceConfig, evaluate_coco, build_coco_results
from mrcnn.prep_notebook import prep_coco_dataset

pp = pprint.PrettyPrinter(indent=2, width=100)
np.set_printoptions(linewidth=100,precision=4,threshold=1000, suppress = True)
start_time = datetime.now().strftime("%m-%d-%Y @ %H:%M:%S")
print()
print('--> Execution started at:', start_time)
print("    Tensorflow Version: {}   Keras Version : {} ".format(tf.__version__,keras.__version__))

Using TensorFlow backend.


Tensorflow Version: 1.8.0   Keras Version : 2.1.6 

--> Execution started at: 10-24-2018 @ 11:42:07
    Tensorflow Version: 1.8.0   Keras Version : 2.1.6 


####  Pass input parameters to argparse

In [2]:
# args = parser.parse_args("--epochs 100 --steps_in_epoch 128  --last_epoch 1264 --batch_size 8  --lr 0.5               --logs_dir train_fcn_adagrad --model /home/kbardool/models/train_mrcnn/shapes20180621T1554/mask_rcnn_shapes_1119.h5 --fcn_model init".split())
# input_parms = "--epochs 100 --steps_in_epoch 100  --last_epoch 1264 --batch_size 25 --lr 0.8 --val_steps 5 --logs_dir train_fcn_adagrad --model /home/kbardool/models/train_mrcnn/shapes20180621T1554/mask_rcnn_shapes_1119.h5 --fcn_model /home/kbardool/models/train_fcn_adagrad/shapes20180709T1732/fcn_shapes_1167.h5"
# input_parms +=" --model     /home/kbardool/models/train_mrcnn/shapes20180621T1554/mask_rcnn_shapes_1119.h5 "
##------------------------------------------------------------------------------------
## Parse command line arguments
##------------------------------------------------------------------------------------
parser = command_line_parser()
input_parms = "--epochs 1 --steps_in_epoch 10  --last_epoch 0 --batch_size 4 --lr 0.001 --val_steps 4 " 
input_parms +="--mrcnn_logs_dir train_mrcnn_coco "
input_parms +="--fcn_logs_dir train_fcn_coco "
input_parms +="--model last "
input_parms +="--fcn_model vgg16 "
input_parms +="--opt adagrad"
# input_parms +="--fcn_model /home/kbardool/models/train_fcn_adagrad/shapes20180709T1732/fcn_shapes_1167.h5"
print(input_parms)

args = parser.parse_args(input_parms.split())
# args = parser.parse_args()

--epochs 1 --steps_in_epoch 10  --last_epoch 0 --batch_size 4 --lr 0.001 --val_steps 4 --mrcnn_logs_dir train_mrcnn_coco --fcn_logs_dir train_fcn_coco --model last --fcn_model vgg16 --opt adagrad


In [3]:
##----------------------------------------------------------------------------------------------
## if debug is true set stdout destination to stringIO
##----------------------------------------------------------------------------------------------            
# debug = False
args.sysout = args.sysout.upper()
if args.sysout == 'FILE':
    sys.stdout = io.StringIO()

# print("    Dataset            : ", args.dataset)
# print("    Logs               : ", args.logs)
# print("    Limit              : ", args.limit)
print("    MRCNN Model        : ", args.model)
print("    FCN Model          : ", args.fcn_model)
print("    MRCNN Log Dir      : ", args.mrcnn_logs_dir)
print("    FCN Log Dir        : ", args.fcn_logs_dir)
print("    Last Epoch         : ", args.last_epoch)
print("    Epochs to run      : ", args.epochs)
print("    Steps in each epoch: ", args.steps_in_epoch)
print("    Validation steps   : ", args.val_steps)
print("    Batch Size         : ", args.batch_size)
print("    Optimizer          : ", args.opt.upper())
# print("    OS Platform        : ", syst)

    MRCNN Model        :  last
    FCN Model          :  vgg16
    MRCNN Log Dir      :  train_mrcnn_coco
    FCN Log Dir        :  train_fcn_coco
    Last Epoch         :  0
    Epochs to run      :  1
    Steps in each epoch:  10
    Validation steps   :  4
    Batch Size         :  4
    Optimizer          :  ADAGRAD


In [4]:
##------------------------------------------------------------------------------------
## setup project directories
##   ROOT_DIR         : Root directory of the project 
##   MODEL_DIR        : Directory to save logs and trained model
##   COCO_MODEL_PATH  : Path to COCO trained weights
##---------------------------------------------------------------------------------
paths = Paths()
paths.display()

##------------------------------------------------------------------------------------
## Build configuration object 
##------------------------------------------------------------------------------------                          
mrcnn_config                    = CocoConfig()
mrcnn_config.NAME               = 'mrcnn'              
mrcnn_config.TRAINING_PATH      = paths.TRAINING_PATH
mrcnn_config.COCO_DATASET_PATH  = paths.COCO_DATASET_PATH 
mrcnn_config.COCO_MODEL_PATH    = paths.COCO_MODEL_PATH   
mrcnn_config.RESNET_MODEL_PATH  = paths.RESNET_MODEL_PATH 
mrcnn_config.VGG16_MODEL_PATH   = paths.VGG16_MODEL_PATH  
mrcnn_config.COCO_CLASSES       = None 
mrcnn_config.DETECTION_PER_CLASS = 200
mrcnn_config.HEATMAP_SCALE_FACTOR = 4

mrcnn_config.BATCH_SIZE         = int(args.batch_size)                  # Batch size is 2 (# GPUs * images/GPU).
mrcnn_config.IMAGES_PER_GPU     = int(args.batch_size)                  # Must match BATCH_SIZE
mrcnn_config.STEPS_PER_EPOCH    = int(args.steps_in_epoch)
mrcnn_config.LEARNING_RATE      = float(args.lr)
mrcnn_config.EPOCHS_TO_RUN      = int(args.epochs)
mrcnn_config.FCN_INPUT_SHAPE    = mrcnn_config.IMAGE_SHAPE[0:2]
mrcnn_config.LAST_EPOCH_RAN     = int(args.last_epoch)
mrcnn_config.WEIGHT_DECAY       = 2.0e-4
mrcnn_config.VALIDATION_STEPS   = int(args.val_steps)
mrcnn_config.REDUCE_LR_FACTOR   = 0.5
mrcnn_config.REDUCE_LR_COOLDOWN = 30
mrcnn_config.REDUCE_LR_PATIENCE = 40
mrcnn_config.EARLY_STOP_PATIENCE= 80
mrcnn_config.EARLY_STOP_MIN_DELTA = 1.0e-4
mrcnn_config.MIN_LR             = 1.0e-10
mrcnn_config.NEW_LOG_FOLDER     = False
mrcnn_config.OPTIMIZER          = args.opt.upper()
mrcnn_config.display()

>>> Initialize Paths
 Linx  Linux

Paths:
-------------------------
COCO_DATASET_PATH              /home/kbardool/MLDatasets/coco2014
COCO_MODEL_PATH                /home/kbardool/PretrainedModels/mask_rcnn_coco.h5
DIR_DATASET                    /home/kbardool/MLDatasets
DIR_PRETRAINED                 /home/kbardool/PretrainedModels
DIR_ROOT                       /home/kbardool/git_projs/mrcnn3/notebooks
DIR_TRAINING                   /home/kbardool/models
FCN_TRAINING_PATH              /home/kbardool/models/train_fcn_coco
FCN_VGG16_MODEL_PATH           /home/kbardool/PretrainedModels/fcn_vgg16_weights_tf_dim_ordering_tf_kernels.h5
RESNET_MODEL_PATH              /home/kbardool/PretrainedModels/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
TRAINING_PATH                  /home/kbardool/models/train_mrcnn_coco
VGG16_MODEL_PATH               /home/kbardool/PretrainedModels/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5



Configuration Parameters:
-------------------------
BACKBO

In [5]:
##------------------------------------------------------------------------------------
## Build Mask RCNN Model in TRAINFCN mode
##------------------------------------------------------------------------------------
# Recreate the model in training mode
try :
    del mrcnn_model,model
    print('delete model is successful')
    gc.collect()
except: 
    pass
KB.clear_session()
mrcnn_model = mrcnn_modellib.MaskRCNN(mode='trainfcn', config=mrcnn_config)

>>> Initialize ModelBase model 
   Mode      :  trainfcn
   Model dir :  /home/kbardool/models/train_mrcnn_coco
>>> ModelBase initialiation complete
>>> ---Initialize MRCNN model, mode:  trainfcn
  set_log_dir(): self.Checkpoint_path: /home/kbardool/models/train_mrcnn_coco/mrcnn20181024T1142/mrcnn_{epoch:04d}.h5 
  set_log_dir(): self.log_dir        : /home/kbardool/models/train_mrcnn_coco/mrcnn20181024T1142 
  set_log_dir(): Last completed epoch (self.epoch): 0 

----------------------------
>>> Resnet Graph 
----------------------------
     Input_image shape : (?, 1024, 1024, 3)
     After ZeroPadding2D  : (?, 1030, 1030, 3) (?, 1030, 1030, 3)
     After Conv2D padding : (?, 512, 512, 64) (?, 512, 512, 64)
     After BatchNorm      : (?, 512, 512, 64) (?, 512, 512, 64)
     C1 Shape: (?, 256, 256, 64) (?, 256, 256, 64)
     C2 Shape:  (?, 256, 256, 256) (?, 256, 256, 256)
     C3 Shape:  (?, 128, 128, 512) (?, 128, 128, 512)
     C4 Shape:  (?, 64, 64, 1024) (?, 64, 64, 1024)
     C

    prob_grid_clipped      :  (?, 256, 256)
    scores_scattered shape :  (4, 81, 200, 3)
    gauss_scores           :  (4, 81, 200, 11)  Name:    cntxt_layer_gt/gt_heatmap_scores:0
    gauss_scores  (FINAL)  :  (4, 81, 200, 11)  Keras tensor  False

    Scatter out the probability distributions based on class --------------
    pt2_ind shape   :  (?, 3)
    prob_grid shape :  (?, 256, 256)
    gauss_scatt     :  (4, 81, 200, 256, 256)

    Reduce MAX based on class ---------------------------------------------
    gaussian_heatmap :  (4, 81, 256, 256) Keras tensor  False
    gauss_heatmap :  (4, 256, 256, 81)  Keras tensor  False

    gt_heatmap                  :  (4, 256, 256, 81) Keras tensor  False
    gt_heatmap_scores           :  (4, 81, 200, 11) Keras tensor  False
    complete
<<<  shape of pred_heatmap   :  (4, 256, 256, 81)  Keras tensor  True
<<<  shape of gt_heatmap     :  (4, 256, 256, 81)  Keras tensor  True

>>> Build MaskRCNN build complete. mode:  trainfcn
>>> MaskRC

In [6]:
##------------------------------------------------------------------------------------
## Load Mask RCNN Model Weight file
##------------------------------------------------------------------------------------
mrcnn_model.load_model_weights(init_with = args.model, exclude = [])   

-----------------------------------------------
 Load Model with init parm: [ last ]
 Exclude layers: 
-----------------------------------------------
 ---> last
>>> find_last checkpoint in :  /home/kbardool/models/train_mrcnn_coco
 Key : > mrcnn <
 Dir names:  ['mrcnn20181024T1142', 'mrcnn20181011T1100']
 Folder:  /home/kbardool/models/train_mrcnn_coco/mrcnn20181024T1142  Checkpoints:  []
 Folder:  /home/kbardool/models/train_mrcnn_coco/mrcnn20181011T1100  Checkpoints:  ['mrcnn_0001.h5', 'mrcnn_0034.h5', 'mrcnn_0071.h5', 'mrcnn_0103.h5']
    find_last():   dir_name: /home/kbardool/models/train_mrcnn_coco/mrcnn20181011T1100
    find_last(): checkpoint: /home/kbardool/models/train_mrcnn_coco/mrcnn20181011T1100/mrcnn_0103.h5
   Last file is : /home/kbardool/models/train_mrcnn_coco/mrcnn20181011T1100/mrcnn_0103.h5
>>> find_last checkpoint in :  /home/kbardool/models/train_mrcnn_coco
 Key : > mrcnn <
 Dir names:  ['mrcnn20181024T1142', 'mrcnn20181011T1100']
 Folder:  /home/kbardool/models/

                                1 (128,)  
                                2 (128,)  
                                3 (128,)  
                               len weight_values: 4   len symbolic_weights: 4
                               i: 0  hdf5 weight shape: (128,)   symb wgt shp: (128,)
                                0 bn3b_branch2a/gamma:0                hdf5 Weights: (128,)  Symbolic Wghts: (128,)
                               i: 1  hdf5 weight shape: (128,)   symb wgt shp: (128,)
                                1 bn3b_branch2a/beta:0                 hdf5 Weights: (128,)  Symbolic Wghts: (128,)
                               i: 2  hdf5 weight shape: (128,)   symb wgt shp: (128,)
                                2 bn3b_branch2a/moving_mean:0          hdf5 Weights: (128,)  Symbolic Wghts: (128,)
                               i: 3  hdf5 weight shape: (128,)   symb wgt shp: (128,)
                                3 bn3b_branch2a/moving_variance:0      hdf5 Weights: (128,)  Symbolic

                               len weight_values: 4   len symbolic_weights: 4
                               i: 0  hdf5 weight shape: (256,)   symb wgt shp: (256,)
                                0 bn4g_branch2b/gamma:0                hdf5 Weights: (256,)  Symbolic Wghts: (256,)
                               i: 1  hdf5 weight shape: (256,)   symb wgt shp: (256,)
                                1 bn4g_branch2b/beta:0                 hdf5 Weights: (256,)  Symbolic Wghts: (256,)
                               i: 2  hdf5 weight shape: (256,)   symb wgt shp: (256,)
                                2 bn4g_branch2b/moving_mean:0          hdf5 Weights: (256,)  Symbolic Wghts: (256,)
                               i: 3  hdf5 weight shape: (256,)   symb wgt shp: (256,)
                                3 bn4g_branch2b/moving_variance:0      hdf5 Weights: (256,)  Symbolic Wghts: (256,)

147 activation_29             Model Layer Name/Type : [('activation_29', <keras.layers.core.Activation object at 

243 bn4q_branch2a             Model Layer Name/Type : [('bn4q_branch2a', <mrcnn.batchnorm_layer.BatchNorm object at 0x7f9081c77ac8>)] 
                              Weights     : ['bn4q_branch2a/gamma:0', 'bn4q_branch2a/beta:0', 'bn4q_branch2a/moving_mean:0', 'bn4q_branch2a/moving_variance:0']
                               Symbolic Weights from Model
                                0 (256,)  
                                1 (256,)  
                                2 (256,)  
                                3 (256,)  
                               len weight_values: 4   len symbolic_weights: 4
                               i: 0  hdf5 weight shape: (256,)   symb wgt shp: (256,)
                                0 bn4q_branch2a/gamma:0                hdf5 Weights: (256,)  Symbolic Wghts: (256,)
                               i: 1  hdf5 weight shape: (256,)   symb wgt shp: (256,)
                                1 bn4q_branch2a/beta:0                 hdf5 Weights: (256,)  Symbolic Wghts:

                               Symbolic Weights from Model
                               len weight_values: 0   len symbolic_weights: 0

337 res5c_branch2b            Model Layer Name/Type : [('res5c_branch2b', <keras.layers.convolutional.Conv2D object at 0x7f9081461d68>)] 
                              Weights     : ['res5c_branch2b/kernel:0', 'res5c_branch2b/bias:0']
                               Symbolic Weights from Model
                                0 (3, 3, 512, 512)  
                                1 (512,)  
                               len weight_values: 2   len symbolic_weights: 2
                               i: 0  hdf5 weight shape: (3, 3, 512, 512)   symb wgt shp: (3, 3, 512, 512)
                                0 res5c_branch2b/kernel:0              hdf5 Weights: (3, 3, 512, 512)  Symbolic Wghts: (3, 3, 512, 512)
                               i: 1  hdf5 weight shape: (512,)   symb wgt shp: (512,)
                                1 res5c_branch2b/bias:0           

   Weights file loaded: /home/kbardool/models/train_mrcnn_coco/mrcnn20181011T1100/mrcnn_0103.h5 
MRCNN  MODEL Load weight file COMPLETE    


In [7]:
mrcnn_model.layer_info()
mrcnn_model.config.display()
print(mrcnn_model.log_dir)



 Inputs:
 -------
 index:  0    input name : input_image:0                              Type: float32           Shape: (?, 1024, 1024, 3)
 index:  1    input name : input_image_meta:0                         Type: float32           Shape: (?, ?)
 index:  2    input name : input_rpn_match:0                          Type: int32             Shape: (?, ?, 1)
 index:  3    input name : input_rpn_bbox:0                           Type: float32           Shape: (?, ?, 4)
 index:  4    input name : input_gt_class_ids:0                       Type: int32             Shape: (?, ?)
 index:  5    input name : input_gt_boxes:0                           Type: float32           Shape: (?, ?, 4)


 Outputs:
 --------
 layer:  0    output name: cntxt_layer/pred_heatmap_norm:0            Type: float32           Shape: (4, 256, 256, 81)
 layer:  1    output name: cntxt_layer/pred_heatmap_scores:0          Type: float32           Shape: (4, 81, 200, 11)
 layer:  2    output name: cntxt_layer_gt/gt_heatmap

In [8]:
##------------------------------------------------------------------------------------
## Build & Load Training and Validation datasets
##------------------------------------------------------------------------------------
# del dataset_train, dataset_val, train_generator
dataset_train = prep_coco_dataset(["train",  "val35k"], mrcnn_config)
# dataset_val   = prep_coco_dataset(["minival"]         , mrcnn_config, generator = False)


loading annotations into memory...
Done (t=12.27s)
creating index...
index created!
 image dir        :  /home/kbardool/MLDatasets/coco2014/train2014
 json_path_dir    :  /home/kbardool/MLDatasets/coco2014/annotations/instances_train2014.json
 number of images :  82081
loading annotations into memory...
Done (t=6.04s)
creating index...
index created!
 image dir        :  /home/kbardool/MLDatasets/coco2014/val2014
 json_path_dir    :  /home/kbardool/MLDatasets/coco2014/annotations/instances_valminusminival2014.json
 number of images :  35185


In [9]:
type(dataset_train)
dest_path = os.path.join(paths.DIR_DATASET, 'coco2014_heatmaps/train_heatmaps')
print(dest_path)

/home/kbardool/MLDatasets/coco2014_heatmaps/train_heatmaps


In [19]:
batch_size = 4
start_from = 100
##--------------------------------------------------------------------------------
## Data generators
##--------------------------------------------------------------------------------
train_generator= data_generator(dataset_train, mrcnn_model.config, shuffle=False,
                                 batch_size=batch_size, image_index = start_from)
build_heatmap_files(mrcnn_model, train_generator, iterations= 5, batch_size=batch_size, dest_path = dest_path)

Starting for  5 iterations - batch size of each iteration: 4


/home/kbardool/anaconda3/envs/TFG/lib/python3.5/site-packages/scipy/ndimage/interpolation.py:616: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


 ==> mrcnn_model: step 0 of 5 iterations, image_id: [101 102 103 104]
   length of train_batch_x: 6  number of things in batch x : (4, 89)
  output: 0 of 4 iterations, image_id: 101
  pr_hm_norm shape   : (256, 256, 81)
  pr_hm_scores shape : (81, 200, 11)
  gt_hm_norm shape   : (256, 256, 81)
  gt_hm_scores shape : (81, 200, 11)
  - output file:  /home/kbardool/MLDatasets/coco2014_heatmaps/train_heatmaps/hm_00101
  output: 1 of 4 iterations, image_id: 102
  pr_hm_norm shape   : (256, 256, 81)
  pr_hm_scores shape : (81, 200, 11)
  gt_hm_norm shape   : (256, 256, 81)
  gt_hm_scores shape : (81, 200, 11)
  - output file:  /home/kbardool/MLDatasets/coco2014_heatmaps/train_heatmaps/hm_00102
  output: 2 of 4 iterations, image_id: 103
  pr_hm_norm shape   : (256, 256, 81)
  pr_hm_scores shape : (81, 200, 11)
  gt_hm_norm shape   : (256, 256, 81)
  gt_hm_scores shape : (81, 200, 11)
  - output file:  /home/kbardool/MLDatasets/coco2014_heatmaps/train_heatmaps/hm_00103
  output: 3 of 4 iterati

In [18]:
def build_heatmap_files(
          mrcnn_model,
          tr_generator, 
          iterations        = 5,
          batch_size        = 2,
          dest_path         = None):
       
    '''
    train_dataset:  Training Dataset objects.

    '''
    assert mrcnn_model.mode == "trainfcn", "Create model in training mode."
    log("Starting for  {} iterations - batch size of each iteration: {}".format(iterations, batch_size))
    
    ##--------------------------------------------------------------------------------
    ## Start main loop
    ##--------------------------------------------------------------------------------
    epoch_idx = 0
    for  epoch_idx in range(iterations) :
        tm_start = time.time()
        
        train_batch_x, train_batch_y = next(tr_generator)
        print(' ==> mrcnn_model: step {} of {} iterations, image_id: {}'.format(epoch_idx, iterations, train_batch_x[1][:,0]))
        print('   length of train_batch_x:', len(train_batch_x), ' number of things in batch x :', train_batch_x[1].shape)
        
#         for i in train_batch_x:
#             print('       ', i.shape)
#         print('length of train_batch_y:', len(train_batch_y))

#         results = get_layer_output_1(mrcnn_model.keras_model, train_batch_x, [0,1,2,3], 1)
        results = mrcnn_model.keras_model.predict(train_batch_x) 
        
#         pr_hm_norm, gt_hm_norm, pr_hm_scores, gt_hm_scores = results[:4]              
        
        for i in range(batch_size):
            print('  output: {} of {} iterations, image_id: {}'.format(i, batch_size, train_batch_x[1][i,0]))
            # print('  pr_hm_norm shape   :', results[0][i].shape)
            # print('  pr_hm_scores shape :', results[1][i].shape)
            # print('  gt_hm_norm shape   :', results[2][i].shape)
            # print('  gt_hm_scores shape :', results[3][i].shape)
            
            filename = dest_path+'/hm_{:05d}'.format(train_batch_x[1][i,0])
            # print('  - output file: ',filename)
            np.savez_compressed(filename, 
                             input_image_meta=train_batch_x[1][i], 
                             pr_hm_norm   = results[0][i],
                             pr_hm_scores = results[1][i],
                             gt_hm_norm   = results[2][i],
                             gt_hm_scores = results[3][i])
            
        tm_stop= time.time()            
        print(' ==> Elapsed time {:.4f}s #        of items in results: {} '.format(tm_stop - tm_start,len(train_batch_x)))

    print('Final : mrcnn_model epoch_idx{}   iterations {}'.format(epoch_idx, iterations))
    return



In [ ]:
dest = 'abcd'
filename = dest+'/{:05d}'.format(0)
print(filename)

In [ ]:

epochs = fcn_model.epoch + epochs_to_run
log("Starting at epoch {} of {} epochs. LR={}\n".format(fcn_model.epoch, epochs, learning_rate))
log("Steps per epochs {} ".format(steps_per_epoch))
log("    Last epoch completed : {} ".format(fcn_model.epoch))
log("    Starting from epoch  : {} for {} epochs".format(fcn_model.epoch, epochs_to_run))
log("    Learning Rate        : {} ".format(learning_rate))
log("    Steps per epoch      : {} ".format(steps_per_epoch))
log("    Batch Size           : {} ".format(batch_size))
log("    Checkpoint Folder    : {} ".format(fcn_model.checkpoint_path))

### Run MRCNN model predictions

     Outputs:
     --------
     layer:  0    output : rpn_class_logits/rpn_class_logits:0        Type: float32           Shape: (?, ?, 2)
     layer:  1    output : rpn_class/rpn_class:0                      Type: float32           Shape: (?, ?, 2)
     layer:  2    output : rpn_bbox/rpn_bbox:0                        Type: float32           Shape: (?, ?, 4)
     layer:  3    output : rpn_proposal_rois/rpn_roi_proposals:0      Type: float32           Shape: (8, ?, ?)
     layer:  4    output : proposal_targets/output_rois:0             Type: float32           Shape: (8, ?, ?)
     layer:  5    output : proposal_targets/target_class_ids:0        Type: int32             Shape: (8, ?)
     layer:  6    output : proposal_targets/target_bbox_deltas:0      Type: float32           Shape: (8, ?, ?)
     layer:  7    output : proposal_targets/roi_gt_boxes:0            Type: float32           Shape: (8, ?, ?)
     layer:  8    output : mrcnn_class_logits/mrcnn_class_logits:0    Type: float32           Shape: (?, 32, 7)
     layer:  9    output : mrcnn_class/mrcnn_class:0                  Type: float32           Shape: (?, 32, 7)
     layer: 10    output : mrcnn_bbox_regression/mrcnn_bbox:0         Type: float32           Shape: (?, 32, 7, 4)
     layer: 11    output : cntxt_layer/pred_heatmap_norm:0            Type: float32           Shape: (8, 128, 128, 7)
     layer: 12    output : cntxt_layer/gt_heatmap_norm:0              Type: float32           Shape: (8, 128, 128, 7)
     layer: 13    output : cntxt_layer/pred_heatmap_scores:0          Type: float32           Shape: (?, ?, ?, ?)
     layer: 14    output : cntxt_layer/gt_heatmap_scores:0            Type: float32           Shape: (?, ?, ?, ?)
        

     Inputs:
     -------
     layer:  0    output : input_pr_hm_norm:0                         Type: float32           Shape: (?, 128, 128, 7)
     layer:  1    output : input_pr_hm_scores:0                       Type: float32           Shape: (?, 7, 32, 11)
     layer:  2    output : input_gt_hm_norm:0                         Type: float32           Shape: (?, 128, 128, 7)
     layer:  3    output : input_gt_hm_scores:0                       Type: float32           Shape: (?, 7, 32, 11)


     Outputs:
     --------
     layer:  0    output : fcn_heatmap_norm/fcn_heatmap_norm:0        Type: float32           Shape: (?, 128, 128, 7)
     layer:  1    output : fcn_scoring/fcn_heatmap_scores:0           Type: float32           Shape: (?, ?, ?, ?)
     layer:  2    output : fcn_heatmap/fcn_heatmap:0                  Type: float32           Shape: (?, 128, 128, 7)
     layer:  3    output : fcn_norm_loss/fcn_norm_loss:0              Type: float32           Shape: (1, 1)     

In [ ]:
results = mrcnn_model.keras_model.predict(train_batch_x)
print('# of items in results:', len(results))

for i in results:
    print( i.shape)


## NPZ Code

In [ ]:
pr_hm_norm, gt_hm_norm, pr_hm_scores, gt_hm_scores = results[11:]                 
print(pr_hm_norm.shape)
print(gt_hm_norm.shape)
print(pr_hm_scores.shape)
print(gt_hm_scores.shape)

In [ ]:
# model.keras_model.summary(line_length=132, positions=[0.30,0.75, .83, 1. ])

In [ ]:
train_batch_x, train_batch_y = next(train_generator)
imgmeta_idx = model.keras_model.input_names.index('input_image_meta')
img_meta    = train_batch_x[imgmeta_idx]

for img_idx in range(config.BATCH_SIZE):
    image_id = img_meta[img_idx,0]
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    print('Image id: ',image_id)
    print('Image meta', img_meta[img_idx])
    print('Classes (1: circle, 2: square, 3: triangle ): ',class_ids)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

In [ ]:
model.layer_info()

In [ ]:
# train_batch_x, train_batch_y = next(train_generator)

model_output = get_layer_output_2(model.keras_model, train_batch_x, 1)
# model_output = get_layer_output_1(model.keras_model, train_batch_x, [ 26], 1)

print(len(model_output))

# rpn_class_loss            = model_output[0]          # layer: 11   shape: (1, 1)
# rpn_bbox_loss             = model_output[1]          # layer: 12   shape: (1, 1)
# mrcnn_class_loss          = model_output[2]          # layer: 13   shape: (1, 1)
# mrcnn_bbox_loss           = model_output[3]          # layer: 14   shape: (1, 1)
# fcn_normalized_loss       = model_output[0]          # layer: 26   shape: (1, 1)

# print(type(output_rois))
for i in model_output:
    print( i.shape)
# print('FCN Normalized Loss is :', fcn_normalized_loss)    


In [ ]:
input_image      =  train_batch_x[0]
input_image_meta =  train_batch_x[1]
input_rpn_match  =  train_batch_x[2]
input_rpn_bbox   =  train_batch_x[3]
input_gt_class_ids = train_batch_x[4]
input_gt_bboxes    = train_batch_x[5]
input_gt_masks     = train_batch_x[6]
print(' Input image shape is :', input_image.shape)
h, w = input_image.shape[1], input_image.shape[2]      #  tf.shape(input_image)[1], tf.shape(input_image)[2]
input_normlzd_gt_bboxes = tf.identity(input_gt_bboxes / [h,w,h,w])

# gt_masks   =  train_batch_x[6]
print(' input_rpn_match    ', input_rpn_match.shape)
print(' input_rpn_bbox     ', input_rpn_bbox.shape)
print(' input_gt_class_ids ', input_gt_class_ids.shape)
print(' input_gt_bboxes    ', input_gt_bboxes.shape)
print(' input_normlzd_gt_bboxes    ', input_normlzd_gt_bboxes.shape)


In [ ]:
np.savez_compressed('../hm_'+str(input_image_meta[0,0]), input_image_meta=input_image_meta[0], 
                 pred_heatmap_norm=model_pred_heatmap_norm[0],
                 pred_heatmap_scores = model_pred_heatmap_scores[0],
                 gt_heatmap_norm = model_gt_heatmap_norm[0], 
                 gt_heatmap_scores = model_gt_heatmap_scores[0])

In [ ]:
loaddata = np.load(dest_path+'/hm_00001.npz')
# input_obj = {}
for i in loaddata.keys():
        print('   ',i, '   ',loaddata[i].shape)
#      input_obj[i]=loaddata[i]
# loaddata.close()    

In [ ]:
type(data)
data.keys()
print(input_image_meta[0,0])
for i in data.keys():
        print(i, data[i].shape)
print(np.all(data['gt_heatmap_norm'] == model_gt_heatmap_norm[0]))  
print(np.all(data['gt_heatmap_scores'] == model_gt_heatmap_scores[0]))  
print(np.all(data['pred_heatmap_norm'] == model_pred_heatmap_norm[0]))  
print(np.all(data['pred_heatmap_scores'] == model_pred_heatmap_scores[0]))  
print(np.all(data['input_image_meta'] == input_image_meta[0]))   

In [ ]:
res = mrcnn_model.keras_model.predict(train_batch_x)

In [ ]:
for i in res:
    print(i.shape)